Assuming you have already got your tip data in netcdf files created using the **`cookie_cutter_tip_csv_to_tip_csv`** notebook, then you should be able to easily convert to freq using the functions contained in **`tip_nc_to_freq_nc.py`**. This particular example is very straightforward. 

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from tip_nc_to_freq_nc import freq_from_tips, calculate_freq

You need to know where your data are stored and have a rain gage network name that you are going to use as a label for the resultant frequency netcdf file.

In [2]:
DATA_DIR = '/home/jsignell/data/ShortCreek/'
RGN = 'Short_Creek'

Now this is proabably the trickiest bit if you don't have your data structured in some reasonable way, then you can walk down from your **`DATA_DIR`** and find all the directories which have files in them that end with **`_tip.nc`**. keep track of these files and the path that points to them. For instance the following cell creates a list of paths that have tip files in them, and a list of lists of filenames. One filename list corresponds to each path.

In [3]:
paths = []
filenames = []
for path, dirs, files in os.walk(DATA_DIR):
    fs = [file for file in files if file.endswith('_tip.nc')]
    if len(fs) > 0:
        paths.append(path)
        filenames.append(fs)

If you are unsure what just happened, just print out the first item in paths, and the first item in filenames. Then run the cell below, making sure that you have the frequency that you want and that the save path is correct.

In [4]:
for path, files in zip(paths, filenames):
    ds = freq_from_tips(
                        path,
                        files,
                        freq='5min',
                        buffer='1H',
                        )
    save_path = '{path}/{RGN}_freq.nc'.format(path=path, RGN=RGN)
    ds.to_netcdf(save_path, format='netCDF4', engine='h5netcdf')

TODO figure out how to force the station to be a string dtype rather than an object.

In [5]:
ds.station

<xarray.DataArray 'station' (station: 6)>
array([b'7436', b'1595', b'7444', b'1549', b'7455', b'1694'], dtype=object)
Coordinates:
    lon           (station) float64 -113.2 -112.8 -113.0 -113.2 -113.0 -112.9
    elev          (station) float64 2.008e+03 1.537e+03 1.525e+03 1.719e+03 ...
    lat           (station) float64 36.47 36.91 36.95 36.95 37.0 36.88
    station_name  (station) |S21 b'Craigs Knoll' b'Moccasin' b'Cane Beds' ...
  * station       (station) object b'7436' b'1595' b'7444' b'1549' b'7455' ...
Attributes:
    cf_role: timeseries_id
    long_name: station ID
    standard_name: station_id

That's it!